In [11]:
import pandas as pd
import numpy as np
import os

In [12]:
# SETTINGS
num_bidders = 6
num_brokers = 3
# -----
num_asset = num_bidders - num_brokers
assert num_brokers < num_bidders

In [13]:
output_dir = "./generator_output"
output_dir_bids = f"{output_dir}/bids"

### Generate Lanes

In [14]:
origins = ["Chicago, IL", "Dallas, TX", "Los Angeles, CA"]
destinations = ["Atlanta, GA", "Boston, MA", "Charlotte, NC", "Denver, CO", "Houston, TX", 
                "Indianapolis, IN", "Jacksonville, FL", "Kansas City, MO", "Las Vegas, NV", 
                "Miami, FL", "Minneapolis, MN", "Nashville, TN", "New Orleans, LA", 
                "New York, NY", "Phoenix, AZ", "Pittsburgh, PA", "Portland, OR", 
                "Salt Lake City, UT", "San Antonio, TX", "Seattle, WA", "St. Louis, MO"]
# Partial distance matrix (expand for all destinations)
distances = {
    "Chicago, IL": [700, 1000, 750, 1000, 1100, 180, 1000, 550, 1750, 1400, 400, 450, 900, 800, 1750, 450, 2100, 1400, 1200, 2000, 300],
    "Dallas, TX": [800, 1550, 1000, 800, 250, 900, 1000, 550, 1200, 1300, 1000, 650, 500, 1550, 1050, 1200, 2000, 1200, 300, 2100, 600],
    "Los Angeles, CA": [2150, 2900, 2400, 1050, 1550, 2050, 2400, 1600, 270, 2700, 1900, 2000, 1900, 2800, 400, 2400, 950, 700, 1350, 1200, 1800]
}



In [15]:
data = []
lane_counter = 1

for o in origins:
    for d, dist in zip(destinations, distances[o]):
        cost = dist * 1.75 * np.random.uniform(0.9, 1.1)
        lane_id = f"Lane-{lane_counter:04d}"
        data.append([lane_id, o, d, dist, "Dry Van", round(cost, 2)])
        lane_counter += 1

lane_df = pd.DataFrame(data, columns=["LaneID","Origin", "Destination", "Distance", "Freight Type", "Historical Cost"])
lane_df.head()

,LaneID,Origin,Destination,Distance,Freight Type,Historical Cost
0,Lane-0001,"Chicago, IL","Atlanta, GA",700,Dry Van,1143.84
1,Lane-0002,"Chicago, IL","Boston, MA",1000,Dry Van,1816.88
2,Lane-0003,"Chicago, IL","Charlotte, NC",750,Dry Van,1184.34
3,Lane-0004,"Chicago, IL","Denver, CO",1000,Dry Van,1832.02
4,Lane-0005,"Chicago, IL","Houston, TX",1100,Dry Van,2010.36


In [16]:
lane_df.to_csv(f"{output_dir}/lane_data.csv", index=False)

### Generate Bids

In [17]:
# Function to generate bid file for a single carrier
def generate_bid_file(lane_df, carrier_id, carrier_type, output_dir_bids):
    bid_data = []
    for _, row in lane_df.iterrows():
        base = row["Distance"] * 1.75
        if carrier_type == "Broker":
            bid_price = base * np.random.uniform(0.875, 1.15) * np.random.uniform(0.9, 1.1)
        else:  # Asset
            bid_price = base * np.random.uniform(0.9, 1.05) * np.random.uniform(0.9, 1.1)
        bid_data.append([
            row["LaneID"], row["Origin"], row["Destination"], row["Distance"], row["Freight Type"],
            carrier_id, carrier_type, round(bid_price, 2)
        ])
    bid_df = pd.DataFrame(bid_data, columns=["LaneID", "Origin", "Destination", "Distance", "Freight Type", "CarrierID", "CarrierType", "BidPrice"])
    bid_df.to_csv(f"{output_dir_bids}/bid_{carrier_id}.csv", index=False)
    return bid_df

In [18]:
# Create carrier list
carriers = []
for i in range(num_brokers):
    carriers.append((f"B{i+1:03d}", "Broker"))
for i in range(num_asset):
    carriers.append((f"A{i+1:03d}", "Asset"))

# Generate bid file for each carrier
for carrier_id, carrier_type in carriers:
    bid_df = generate_bid_file(lane_df, carrier_id, carrier_type, output_dir_bids)
    print(f"Generated bid file for {carrier_id}: {output_dir_bids}/bid_{carrier_id}.csv")


Generated bid file for B001: ./generator_output/bids/bid_B001.csv
Generated bid file for B002: ./generator_output/bids/bid_B002.csv
Generated bid file for B003: ./generator_output/bids/bid_B003.csv
Generated bid file for A001: ./generator_output/bids/bid_A001.csv
Generated bid file for A002: ./generator_output/bids/bid_A002.csv
Generated bid file for A003: ./generator_output/bids/bid_A003.csv


In [19]:
# Optional: Preview one bid file
sample_bid = pd.read_csv(f"{output_dir_bids}/bid_B001.csv")
print("\nSample bid file (B001):")
print(sample_bid.head())


Sample bid file (B001):
      LaneID       Origin    Destination  Distance Freight Type CarrierID  \
0  Lane-0001  Chicago, IL    Atlanta, GA       700      Dry Van      B001   
1  Lane-0002  Chicago, IL     Boston, MA      1000      Dry Van      B001   
2  Lane-0003  Chicago, IL  Charlotte, NC       750      Dry Van      B001   
3  Lane-0004  Chicago, IL     Denver, CO      1000      Dry Van      B001   
4  Lane-0005  Chicago, IL    Houston, TX      1100      Dry Van      B001   

  CarrierType  BidPrice  
0      Broker   1103.41  
1      Broker   1932.08  
2      Broker   1473.04  
3      Broker   1536.06  
4      Broker   1907.49  
